# Training with a custom implementation of ResNet

In [28]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np 
import pandas as pd
import pickle

import matplotlib.pyplot as plt

from skimage import io

# tqdm is a library that enables you to visualize the progress of a for loop by displaying a configurable progress bar
from tqdm.notebook import tqdm
tqdm().pandas();

0it [00:00, ?it/s]

## 1. Loading data

In [2]:
sample_df = pd.read_pickle(r"./data/images.pkl")

## 2. Split into training and validation

In [23]:
images_arrays = sample_df.img_array.to_numpy()
X = np.zeros(shape = (150000,3,50,50), dtype = np.int8)
y = sample_df.target.to_numpy()

for index in tqdm(range(150000)):
    X[index] = images_arrays[index].reshape((3,50,50))

  0%|          | 0/150000 [00:00<?, ?it/s]

In [24]:
#shuffle data and split data to generate validation data
shuffler = np.random.RandomState(seed=42).permutation(len(X))
X_shf = X[shuffler]
y_shf = y[shuffler]


# split 60 / 20 / 20
X_train=X_shf[0:90000]
y_train=y_shf[0:90000]
X_val=X_shf[90000:120000]
y_val=y_shf[90000:120000]
X_test=X_shf[120000:150000]
y_test=y_shf[120000:150000]

In [25]:
#save for future use without having to reload from the images folders
np.save('data/X_test.npy', X_test)
np.save('data/y_test.npy', y_test)
np.save('data/X_train.npy', X_train)
np.save('data/y_train.npy', y_train)
np.save('data/X_val.npy', X_val)
np.save('data/y_val.npy', y_val) 

In [29]:
print('script running in '+os.path.abspath("."))
X_test = np.load('./data/X_test.npy')
y_test = np.load('./data/y_test.npy')
X_train = np.load('./data/X_train.npy')
y_train = np.load('./data/y_train.npy')
X_val = np.load('./data/X_val.npy')
y_val = np.load('./data/y_val.npy')

script running in c:\Users\anton\Documents\ML Projects\BreastHistopathologyWithPytorch


## 3. Using ResNet 18 with the Breast Histopathology dataset

In [30]:
from models.resnet import ResNet18